<a href="https://colab.research.google.com/github/nithinreganti/Understanding-the-pricing-model-of-Uber-and-Lyft-through-Exploratory-Data-Analysis-and-Machine-Learn/blob/master/UberLyft_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/5/58/Uber_logo_2018.svg/330px-Uber_logo_2018.svg.png'/>
<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Lyft_logo.svg/330px-Lyft_logo.svg.png' />

## Introduction
<b>Uber</b> is an American multinational ridesharing company offering services that include peer-to-peer ridesharing, ride service hailing, food delivery, and a micromobility system with electric bikes and scooters. The company is based in San Francisco and has operations in over 785 metropolitan areas worldwide. Its platforms can be accessed via its websites and mobile apps.<br><br>
<b>Lyft</b> is a ridesharing company based in San Francisco, California and operating in 640 cities in the United States and 9 cities in Canada. It develops, markets, and operates the Lyft mobile app, offering car rides, scooters, a bicycle-sharing system, and a food delivery services. Lyft is the second-largest ridesharing company in the United States with a 28% market share after Uber, according to Second Measure.<br><br>
<div style="text-align: right"> Source: <b>Wikipedia</b> </div>

---

The following notebook is an attempt at understanding the pricing model of Uber and Lyft cab rides. For this, we will be using a dataset which records bookings of Uber and Lyft cab rides in the city of Boston during November and December 2018. The dataset is publicly available on <a href='https://www.kaggle.com/brllrb/uber-and-lyft-dataset-boston-ma'>Kaggle</a>.<br>

We wish to derive insights about how factors such as distance, cab type, time of day and weather conditions affect prices of cab rides and compare the differences between the two companies. We have also implemented a regression model to predict prices and a classification model to classify based on the company. These models also give us an insight into the feature importances and coefficients which give us additional information about how individual features affect the pricing of a cab ride.<br>

Exploartory Data Analysis has been carried out using Plotly visualization library which produces interactive visualizations. You can hover over the plots for more information and also control the plot using the legend in the top right corner.  

**<i>Note: Plotly version 3.10.0 and cufflinks version 0.13.0 are required to run these interactive visualizations without any errors. If you do not have the required libraries, please install them and restart this notebook. Or, use the following link to view this notebook with interactive visualizations rendered - </i>**
https://nbviewer.jupyter.org/github/nithinreganti/Understanding-the-pricing-model-of-Uber-and-Lyft-through-Exploratory-Data-Analysis-and-Machine-Learn/blob/master/UberLyft_.ipynb

## Importing Libraries

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from datetime import datetime
import sqlite3
import plotly.offline as py
import cufflinks as cf
from plotly.offline import download_plotlyjs, iplot, plot, init_notebook_mode
from plotly import tools
import plotly.graph_objs as go
init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore')

## Data Import and Exploration

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
try:
    conn=sqlite3.connect("CabData.db")
    print("Connection successful")
    
except Error as e:
    print(e)
    
#Loading two datasets into database.
df_sql = pd.read_csv('/content/drive/My Drive/Python-Uberlyft/rideshare_kaggle.csv')
geo_sql = pd.read_csv('/content/drive/My Drive/Python-Uberlyft/geolocations.csv')
df_sql.to_sql('Cabdetails',conn, index=False,if_exists='replace')
geo_sql.to_sql('geoLocations',conn, index=False,if_exists='replace')

#Working with selected features which are important to this project 
#and retrieving only those records which has target(price) variable as not null.
query = query = '''SELECT distance, cab_type, timestamp, source, destination, price, surge_multiplier, product_id,
         name, cloudCover, temperature, windSpeed, uvIndex, precipIntensityMax, precipIntensity,
         precipProbability, hour, day, loc.latitude as source_latitude,
         loc.longitude as source_longitude,loc1.latitude as destination_latitude, loc1.longitude as destination_longitude
         from  Cabdetails cab,geoLocations loc, geoLocations loc1 where loc.Place = cab.source 
         and loc1.Place = cab.destination and cab.price is not null'''
df = pd.read_sql_query(query, conn)

conn.close()

Connection successful


In [0]:
df.head()

,distance,cab_type,timestamp,source,destination,price,surge_multiplier,product_id,name,cloudCover,temperature,windSpeed,uvIndex,precipIntensityMax,precipIntensity,precipProbability,hour,day,source_latitude,source_longitude,destination_latitude,destination_longitude
0,0.44,Lyft,1.544953e+09,Haymarket Square,North Station,5.0,1.0,lyft_line,Shared,0.72,42.34,8.66,0,0.1276,0.0000,0.0,9,16,42.3638,-71.0585,42.3657,-71.061
1,0.44,Lyft,1.543284e+09,Haymarket Square,North Station,11.0,1.0,lyft_premier,Lux,1.00,43.58,11.98,0,0.1300,0.1299,1.0,2,27,42.3638,-71.0585,42.3657,-71.061
2,0.44,Lyft,1.543367e+09,Haymarket Square,North Station,7.0,1.0,lyft,Lyft,0.03,38.33,7.33,0,0.1064,0.0000,0.0,1,28,42.3638,-71.0585,42.3657,-71.061
3,0.44,Lyft,1.543554e+09,Haymarket Square,North Station,26.0,1.0,lyft_luxsuv,Lux Black XL,0.00,34.38,5.28,0,0.0000,0.0000,0.0,4,30,42.3638,-71.0585,42.3657,-71.061
4,0.44,Lyft,1.543463e+09,Haymarket Square,North Station,9.0,1.0,lyft_plus,Lyft XL,0.44,37.44,9.14,0,0.0001,0.0000,0.0,3,29,42.3638,-71.0585,42.3657,-71.061


In [0]:
#Rows, columns
df.shape

(531585, 22)

In [0]:
#Columns with number of records and their data types.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531585 entries, 0 to 531584
Data columns (total 22 columns):
distance                 531585 non-null float64
cab_type                 531585 non-null object
timestamp                531585 non-null float64
source                   531585 non-null object
destination              531585 non-null object
price                    531585 non-null float64
surge_multiplier         531585 non-null float64
product_id               531585 non-null object
name                     531585 non-null object
cloudCover               531585 non-null float64
temperature              531585 non-null float64
windSpeed                531585 non-null float64
uvIndex                  531585 non-null int64
precipIntensityMax       531585 non-null float64
precipIntensity          531585 non-null float64
precipProbability        531585 non-null float64
hour                     531585 non-null int64
day                      531585 non-null int64
source_latitude   

In [0]:
#Descriptive Stastics of numerical features
df.describe()

,distance,timestamp,price,surge_multiplier,cloudCover,temperature,windSpeed,uvIndex,precipIntensityMax,precipIntensity,precipProbability,hour,day,source_latitude,source_longitude,destination_latitude,destination_longitude
count,531585.000000,5.315850e+05,531585.000000,531585.000000,531585.000000,531585.000000,531585.000000,531585.000000,531585.000000,531585.000000,531585.000000,531585.000000,531585.000000,531585.000000,531585.000000,531585.000000,531585.000000
mean,2.220454,1.544045e+09,16.628456,1.013666,0.685772,39.578816,6.181071,0.248852,0.037312,0.008868,0.145364,11.615204,17.782409,42.355771,-56.846060,42.355772,-56.849269
std,1.179999,6.887889e+05,9.413309,0.089045,0.358753,6.723335,3.146441,0.473861,0.055178,0.026820,0.328266,6.949588,9.990105,0.008380,42.655766,0.008380,42.651490
min,0.020000,1.543204e+09,2.500000,1.000000,0.000000,18.910000,0.450000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,42.339800,-71.105400,42.339800,-71.105400
25%,1.220000,1.543444e+09,9.000000,1.000000,0.370000,36.450000,3.410000,0.000000,0.000000,0.000000,0.000000,6.000000,13.000000,42.350500,-71.089200,42.350500,-71.089200
50%,2.270000,1.543737e+09,13.500000,1.000000,0.820000,40.490000,5.910000,0.000000,0.000400,0.000000,0.000000,12.000000,17.000000,42.355900,-71.061000,42.355900,-71.061000
75%,3.000000,1.544827e+09,22.500000,1.000000,1.000000,43.580000,8.410000,0.000000,0.091600,0.000000,0.000000,18.000000,28.000000,42.364400,-71.055000,42.364400,-71.055000
max,7.860000,1.545161e+09,97.500000,3.000000,1.000000,57.220000,15.000000,2.000000,0.145900,0.144700,1.000000,23.000000,30.000000,42.365700,71.064300,42.365700,71.064300


In [0]:
#Overview of categorical features.
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
for col in cat_cols:
    print('Column Name: ', col, '\n')
    print('Number of unique values: ', df[col].nunique(), '\n')
    print('Unique values: ', df[col].unique())
    print('\n----------------------------------------\n')

Column Name:  cab_type 

Number of unique values:  2 

Unique values:  ['Lyft' 'Uber']

----------------------------------------

Column Name:  source 

Number of unique values:  11 

Unique values:  ['Haymarket Square' 'North End' 'North Station' 'Beacon Hill'
 'Boston University' 'Fenway' 'Theatre District' 'West End'
 'Financial District' 'South Station' 'Northeastern University']

----------------------------------------

Column Name:  destination 

Number of unique values:  11 

Unique values:  ['North Station' 'West End' 'Haymarket Square' 'South Station'
 'Northeastern University' 'Fenway' 'Theatre District' 'Beacon Hill'
 'North End' 'Financial District' 'Boston University']

----------------------------------------

Column Name:  product_id 

Number of unique values:  12 

Unique values:  ['lyft_line' 'lyft_premier' 'lyft' 'lyft_luxsuv' 'lyft_plus' 'lyft_lux'
 '6f72dfc5-27f1-42e8-84db-ccc7a75f6969'
 '6c84fd89-3f11-4782-9b50-97c468b19529'
 '55c66225-fbe7-4fd5-9072-eab1ece5e23e'

In [0]:
#Columns with their missing values.
df.isnull().sum()

distance                 0
cab_type                 0
timestamp                0
source                   0
destination              0
price                    0
surge_multiplier         0
product_id               0
name                     0
cloudCover               0
temperature              0
windSpeed                0
uvIndex                  0
precipIntensityMax       0
precipIntensity          0
precipProbability        0
hour                     0
day                      0
source_latitude          0
source_longitude         0
destination_latitude     0
destination_longitude    0
dtype: int64

## Visualizations

In [0]:
def layout_details(title, xaxis_title=None, yaxis_title=None):
    '''Add layout details such as Title, X-axis title and Y-axis title.'''
    layout = go.Layout(title=title,
                       xaxis=dict(title=xaxis_title),
                       yaxis=dict(title=yaxis_title)
                      )
    
    return layout

def display_trace(data, layout):
    '''Display visualization with chosen data and layout.'''
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)
    
def plot_bar_chart(x, y, colors=None, name=None, opacity=None):
    '''Plot bar chart.'''
    bar_trace = go.Bar(x=x,
                       y=y,
                       marker_color=colors,
                       opacity=opacity,
                       name=name
                      )
    return bar_trace

def plot_pie_chart(values, labels, domain, textinfo, hole, name=None):
    '''Plot pie/donut chart.'''
    pie_trace = go.Pie(values=values,
                       labels=labels,
                       domain=domain,
                       textinfo=textinfo,
                       hole=hole,
                       name=name
                      )
    
    return pie_trace

def plot_line_chart(x, y, colors=None, name=None, opacity=None):
    '''Plot line chart.'''
    line_trace = go.Scatter(x=x,
                            y=y,
                            marker_color=colors,
                            name=name,
                            opacity=opacity
                           )
    
    return line_trace

def plot_scatter_chart(x, y, colors=None, name=None, opacity=None):
    '''Plot scatter chart.'''
    scatter_trace = go.Scatter(x=x,
                            y=y,
                            marker_color=colors,
                            name=name,
                            opacity=opacity,
                            mode='markers',
                           )
    
    return scatter_trace

def plot_scatter_line_chart(x, y, colors=None, name=None, opacity=None):
    '''Plot scatter chart.'''
    scatter_line_trace = go.Scatter(x=x,
                            y=y,
                            marker_color=colors,
                            name=name,
                            opacity=opacity,
                            mode='lines',
                           )
    
    return scatter_line_trace

In [0]:
token = 'pk.eyJ1IjoiYXAxNDk1IiwiYSI6ImNrM2dwa3lzcjA0Y3AzaWx1aGxjb3VwNTIifQ.qnk9bZu8o0RWFRy-qSsEgA'
hs_count = len(df[df['source'] == 'Haymarket Square'])
bb_count = len(df[df['source'] == 'Back Bay'])
ne_count = len(df[df['source'] == 'North End'])
ns_count = len(df[df['source'] == 'North Station'])
bh_count = len(df[df['source'] == 'Beacon Hill'])
bu_count = len(df[df['source'] == 'Boston University'])
f_count = len(df[df['source'] == 'Fenway'])
ss_count = len(df[df['source'] == 'South Station'])
td_count = len(df[df['source'] == 'Theatre District'])
we_count = len(df[df['source'] == 'West End'])
fd_count = len(df[df['source'] == 'Financial District'])
nu_count = len(df[df['source'] == 'Northeastern University'])

trace = go.Scattermapbox(
        lat=df['source_latitude'].unique().tolist(),
        lon=df['source_longitude'].unique().tolist(),
        mode='markers',
        marker=go.scattermapbox.Marker(size=11,
                                       color='orange',
                                       opacity=0.8
                                      ),
        text=['Haymarket Square<br>'+str(hs_count), 'Back Bay<br>'+str(bb_count), 
              'North End<br>'+str(ne_count), 'North Station<br>'+str(ns_count), 
              'Beacon Hill<br>'+str(bh_count),'Boston University<br>'+str(bu_count), 
              'Fenway<br>'+str(f_count), 'South Station<br>'+str(ss_count), 
              'Theatre District<br>'+str(td_count), 'West End<br>'+str(we_count),
              'Financial District<br>'+str(fd_count), 'Northeastern University<br>'+str(nu_count)
             ]
    )

layout = go.Layout(autosize=True,
                   title='<b>Uber & Lyft Booking Locations in Boston</b>',
                   hovermode='closest',
                   mapbox=go.layout.Mapbox(accesstoken=token,
                                           bearing=0,
                                           center=go.layout.mapbox.Center(lat=42.3469,
                                                                          lon=-71.0661
                                                                         ),
                                            pitch=0,
                                            zoom=12
                                          ),
                )

fig = go.Figure(data=[trace], layout=layout)
py.iplot(fig)

In [0]:
colors = ['black', 'deeppink']
x = df['cab_type'].value_counts().index.values
y = df['cab_type'].value_counts().values
opacity=0.8
trace1 = plot_bar_chart(x, y, colors, opacity=opacity)
layout = layout_details('<b>Count of Rides by Company</b>', '<b><i>Company</b></i>', '<b><i>Number of Rides</b></i>')
display_trace([trace1], layout)

The dataset contains **330,568** records for **Uber** bookings while **307,408** records for **Lyft** bookings. Note that Uber has **23,160** bookings more than Lyft.
<br><br>
*Hover over the visalization to get exact values.*<br><br>

In [0]:
uber_df = df[df['cab_type'] == 'Uber']
lyft_df = df[df['cab_type'] == 'Lyft']

textinfo='value+percent'
hole=0.4

values_uber=uber_df.groupby(by='name')['name'].count().values
labels_uber=uber_df.groupby(by='name')['name'].count().index.values
domain_uber=dict(x=[0, 0.5], y=[1, 0.3])
name='Uber'
trace2a = plot_pie_chart(values_uber, labels_uber, domain_uber, textinfo, hole, name)

values_lyft=lyft_df.groupby(by='name')['name'].count().values
labels_lyft=lyft_df.groupby(by='name')['name'].count().index.values
domain_lyft=dict(x=[0.55, 1], y=[1, 0.3])
name='Lyft'
trace2b = plot_pie_chart(values_lyft, labels_lyft, domain_lyft, textinfo, hole, name)

layout = layout_details('<b>Count of Rides by Cab Classes by Company<br>Uber | Lyft</b>')
display_trace([trace2a, trace2b], layout)

The above visualization shows the split of bookings based on cab classes. Since the bookings we made using a controlled environment, all the classes have more or less the same number of bookings. **~55,000** bookings in each cab class for Uber while **~51,000** bookings in each cab class for Lyft.
<br><br>
*Hover over the visualization for more information and control the visualization using the legend in the right corner.*
<br><br>

In [0]:
fig = tools.make_subplots(rows=1, cols=2)

x=uber_df.groupby(by='name')['price'].sum().index.values
y=uber_df.groupby(by='name')['price'].sum().values
name='Uber'
color='black'
opacity=0.8
trace3a = plot_bar_chart(x, y, color, name, opacity)

x=lyft_df.groupby(by='name')['price'].sum().index.values
y=lyft_df.groupby(by='name')['price'].sum().values
name='Lyft'
color='deeppink'
opacity=0.8
trace3b = plot_bar_chart(x, y, color, name)

fig.append_trace(trace3a, 1,1)
fig.append_trace(trace3b, 1,2)
fig['layout'].update(title='<b>Total Sales by Company based on Cab Classes</b>',
                     xaxis=dict(automargin=True),
                     yaxis=dict(title='<b><i>Sales in USD</b></i>')
                    )
py.iplot(fig)

Uber tends to make more money on their economy rides compared to Lyft. Lyft tends to make more money than Uber on their premium rides. Although, number of Uber bookings are more than Lyft, the total sales of Lyft's premium rides are greater than Uber's premium rides.
<br>
- Black is to Lux Black
- Black SUV is to Lux Black XL
- UberPool is to Shared
- UberX is to Lyft
- UberXL is to Lyft XL
- WAV is to Lux
<br>

*Hover over the bars to get exact sales values.*

### Distance vs Price

In [0]:
x=uber_df.groupby(by='distance')['price'].mean().index.values
y=uber_df.groupby(by='distance')['price'].mean().values
name='Uber'
color='black'
opacity=0.8
trace4a = plot_line_chart(x, y, color, name, opacity)

x=lyft_df.groupby(by='distance')['price'].mean().index.values
y=lyft_df.groupby(by='distance')['price'].mean().values
name='Lyft'
color='deeppink'
opacity=0.8
trace4b = plot_line_chart(x, y, color, name, opacity)

layout = layout_details('<b>Average Cost of Ride by Distance</b>', '<b><i>Distance in miles</i></b>', 
                        '<b><i>Cost in USD</i></b>')
display_trace([trace4a, trace4b], layout)

For smaller distances, there is no significant difference in pricing between the two companies. However, as the distance increases, the difference in pricing between the two companies also increase. Overall, Lyft tends to charge more than Uber.
<br><br>
*Hover over the visualization to compare the prices between Uber and Lyft for a particular distance.*<br><br>

In [0]:
x=uber_df[uber_df['name'] == 'UberX'].groupby(by='distance')['price'].mean().index.values
y=uber_df[uber_df['name'] == 'UberX'].groupby(by='distance')['price'].mean().values
name='UberX'
color='black'
opacity=0.8
trace5a = plot_line_chart(x, y, color, name, opacity)

x=lyft_df[lyft_df['name'] == 'Lyft'].groupby(by='distance')['price'].mean().index.values
y=lyft_df[lyft_df['name'] == 'Lyft'].groupby(by='distance')['price'].mean().values
name='Lyft(Economy)'
color='deeppink'
opacity=0.8
trace5b = plot_line_chart(x, y, color, name, opacity)

layout = layout_details('<b>Average Cost of Ride by Distance for Economy Rides(4 seater) </b>', 
                        '<b><i>Distance in miles</i></b>', 
                        '<b><i>Cost in USD</i></b>')
display_trace([trace5a, trace5b], layout)

For economy rides, Uber tends to charge more than Lyft irrespective of the booking distance.
<br><br>
*Hover over the visualization to compare the prices between Uber and Lyft for a particular distance.*<br><br>

In [0]:
x=uber_df[uber_df['name'] == 'UberXL'].groupby(by='distance')['price'].mean().index.values
y=uber_df[uber_df['name'] == 'UberXL'].groupby(by='distance')['price'].mean().values
name='Uber XL'
color='black'
opacity=0.8
trace7a = plot_line_chart(x, y, color, name, opacity)

x=lyft_df[lyft_df['name'] == 'Lyft XL'].groupby(by='distance')['price'].mean().index.values
y=lyft_df[lyft_df['name'] == 'Lyft XL'].groupby(by='distance')['price'].mean().values
name='Lyft XL'
color='deeppink'
opacity=0.8
trace7b = plot_line_chart(x, y, color, name, opacity)

layout = layout_details('<b>Average Cost of Ride by Distance for Economy XL Rides(6 seater)</b>', 
                        '<b><i>Distance in miles</i></b>', 
                        '<b><i>Cost in USD</i></b>')
display_trace([trace7a, trace7b], layout)

For economy XL rides, there is no significant difference in charges between the two companies for most distances.
<br><br>
*Hover over the visualization to compare the prices between Uber and Lyft for a particular distance.*<br><br>

In [0]:
x=uber_df[uber_df['name'] == 'UberPool'].groupby(by='distance')['price'].mean().index.values
y=uber_df[uber_df['name'] == 'UberPool'].groupby(by='distance')['price'].mean().values
name='Uber Pool'
color='black'
opacity=0.8
trace6a = plot_line_chart(x, y, color, name, opacity)

x=lyft_df[lyft_df['name'] == 'Shared'].groupby(by='distance')['price'].mean().index.values
y=lyft_df[lyft_df['name'] == 'Shared'].groupby(by='distance')['price'].mean().values
name='Lyft Shared'
color='deeppink'
opacity=0.8
trace6b = plot_line_chart(x, y, color, name, opacity)

layout = layout_details('<b>Average Cost of Ride by Distance for Shared Rides</b>', 
                        '<b><i>Distance in miles</i></b>', 
                        '<b><i>Cost in USD</i></b>')
display_trace([trace6a, trace6b], layout)

Lyft charges significantly lower than Uber when it comes to shared rides irrespective of the distance.
<br><br>
*Hover over the visualization to compare the prices between Uber and Lyft for a particular distance.*<br><br>

In [0]:
x=uber_df[uber_df['name'] == 'Black'].groupby(by='distance')['price'].mean().index.values
y=uber_df[uber_df['name'] == 'Black'].groupby(by='distance')['price'].mean().values
name='Uber Black'
color='black'
opacity=0.8
trace8a = plot_line_chart(x, y, color, name, opacity)

x=lyft_df[lyft_df['name'] == 'Lux Black'].groupby(by='distance')['price'].mean().index.values
y=lyft_df[lyft_df['name'] == 'Lux Black'].groupby(by='distance')['price'].mean().values
name='Lyft Black'
color='deeppink'
opacity=0.8
trace8b = plot_line_chart(x, y, color, name, opacity)

layout = layout_details('<b>Average Cost of Ride by Distance for Premium Rides(4 seater)</b>', 
                        '<b><i>Distance in miles</i></b>', 
                        '<b><i>Cost in USD</i></b>')
display_trace([trace8a, trace8b], layout)

Lyft's premium 4 seater ride charges are significantly higher than Uber's premium 4 seater rides across all distances.
<br><br>
*Hover over the visualization to compare the prices between Uber and Lyft for a particular distance.*<br><br>

In [0]:
x=uber_df[uber_df['name'] == 'Black SUV'].groupby(by='distance')['price'].mean().index.values
y=uber_df[uber_df['name'] == 'Black SUV'].groupby(by='distance')['price'].mean().values
name='Uber Black XL'
color='black'
opacity=0.8
trace9a = plot_line_chart(x, y, color, name, opacity)

x=lyft_df[lyft_df['name'] == 'Lux Black XL'].groupby(by='distance')['price'].mean().index.values
y=lyft_df[lyft_df['name'] == 'Lux Black XL'].groupby(by='distance')['price'].mean().values
name='Lyft Black XL'
color='deeppink'
opacity=0.8
trace9b = plot_line_chart(x, y, color, name, opacity)

layout = layout_details('<b>Average Cost of Ride by Distance for Premium Rides(6 seater)</b>', 
                        '<b><i>Distance in miles</i></b>', 
                        '<b><i>Cost in USD</i></b>')
display_trace([trace9a, trace9b], layout)

The same can be said for Lyft's premium 6 seater rides. They charge slightly higher than Uber's 6 seater rides across all distances.
<br><br>
*Hover over the visualization to compare the prices between Uber and Lyft for a particular distance.*

**Overall Lyft appears to cost more than Uber for similar distances. But digging down further, we found that Uber is costlier than Lyft with economy rides while Lyft charges more than Uber for similar distances on its premium rides.**

### Time vs Price

In [0]:
x=df.groupby(by='day')['price'].mean().index.values
y=df.groupby(by='day')['price'].mean().values
trace10 = plot_line_chart(x, y)

layout = layout_details('<b>Average Cost of Ride by Day</b>', 
                        '<b><i>Day</i></b>', 
                        '<b><i>Cost in USD</i></b>')
display_trace([trace10], layout)

Although the graph shows major ups and downs, there is no drastic increase/decrease in average cost of a ride based on a particular day. The highest cost is <b>\$16.79</b> while the lowest is <b>\$16.41</b>. <br><br>

In [0]:
x=df.groupby(by='hour')['price'].mean().index.values
y=df.groupby(by='hour')['price'].mean().values
trace11 = plot_line_chart(x, y)

layout = layout_details('<b>Average Cost of Ride by Hour</b>', 
                        '<b><i>Hour</i></b>', 
                        '<b><i>Cost in USD</i></b>')
display_trace([trace11], layout)

The same can be said about this graph as well. There is no drastic increase/decrease in average cost of a ride based on a particular hour in a day. The highest cost is <b>\$16.60</b> while the lowest is <b>\$16.48</b>. <br><br>

In [0]:
df['distance'].describe()

count    531585.000000
mean          2.220454
std           1.179999
min           0.020000
25%           1.220000
50%           2.270000
75%           3.000000
max           7.860000
Name: distance, dtype: float64

In [0]:
less3_df = df[df['distance'] <= 3]
less6_df = df[(df['distance'] <= 6) & (df['distance'] > 3)]
less15_df = df[df['distance'] > 6]

x=less3_df.groupby(by='hour')['price'].mean().index.values
y=less3_df.groupby(by='hour')['price'].mean().values
name='Less than 3 miles'
trace12a = plot_line_chart(x, y, name=name)

x=less6_df.groupby(by='hour')['price'].mean().index.values
y=less6_df.groupby(by='hour')['price'].mean().values
name='Between 3 & 6 miles'
trace12b = plot_line_chart(x, y, name=name)

x=less15_df.groupby(by='hour')['price'].mean().index.values
y=less15_df.groupby(by='hour')['price'].mean().values
name='Greater than 6 miles'
trace12c = plot_line_chart(x, y, name=name)

layout = layout_details('<b>Average Cost of Ride by Hour on varying Distances</b>', 
                        '<b><i>Hour</i></b>', 
                        '<b><i>Cost in USD</i></b>')
display_trace([trace12a, trace12b, trace12c], layout)

Investigating further into how time of day affects prices, it can be noted from the above visualization that for distances greater than 6 miles, the average cost of a ride varies significantly throughout the day.<br>
**An interesting observation from the above visualization is that the highest increase in average price for distances greater than 6 miles occurs during the time period of 7am to 8am which is often considered as the peak hour where employees are commuting to their offices.**<br><br>
*Hover over the visualization to compare the cost across the different distances and times of day*. <br><br>

In [0]:
x=less15_df[less15_df['cab_type'] == 'Uber'].groupby(by='hour')['price'].mean().index.values
y=less15_df[less15_df['cab_type'] == 'Uber'].groupby(by='hour')['price'].mean().values
name='Uber'
color='black'
opacity=0.8
trace13a = plot_line_chart(x, y, color, name, opacity)

x=less15_df[less15_df['cab_type'] == 'Lyft'].groupby(by='hour')['price'].mean().index.values
y=less15_df[less15_df['cab_type'] == 'Lyft'].groupby(by='hour')['price'].mean().values
name='Lyft'
color='deeppink'
opacity=0.8
trace13b = plot_line_chart(x, y, color, name, opacity)

layout = layout_details('<b><b>Average Cost of Ride by Hour for Distances > 6 miles based on Company</b></b>', 
                        '<b><i>Hour</i></b>', 
                        '<b><i>Cost in USD</i></b>')
display_trace([trace13a, trace13b], layout)

From the above visualization, it is evident that Lyft tends to modify the price of a cab ride based on time of day more than Uber.

*Hover over the visualization to compare the cost between Uber and Lyft.* 
<br><br>

## External Factors vs Price
- Temperature
- Wind Speed
- Precipitation

In [0]:
x=less3_df[less3_df['distance'] == 2.32].groupby(by='temperature')['price'].mean().index.values
y=less3_df[less3_df['distance'] == 2.32].groupby(by='temperature')['price'].mean().values
name='2.32 miles'
trace14a = plot_scatter_chart(x, y, name=name)

x=less6_df[less6_df['distance'] == 3.05].groupby(by='temperature')['price'].mean().index.values
y=less6_df[less6_df['distance'] == 3.05].groupby(by='temperature')['price'].mean().values
name='3.05 miles'
trace14b = plot_scatter_chart(x, y, name=name)

x=less15_df[less15_df['distance'] == 7.46].groupby(by='temperature')['price'].mean().index.values
y=less15_df[less15_df['distance'] == 7.46].groupby(by='temperature')['price'].mean().values
name='7.46 miles'
trace14c = plot_scatter_chart(x, y, name=name)

layout = layout_details('<b>Average Cost of Ride based on Temperature</b>', 
                        '<b><i>Temperature in Fahrenheit</i></b>', 
                        '<b><i>Cost in USD</i></b>')
display_trace([trace14a, trace14b, trace14c], layout)

The above visualization suggests that the cost of a ride in either Uber or Lyft depends on the temperature at the time of booking for distances greater than 6 miles. While the data points for lower distances are clustered close together, the data points for a higher distance appears to be more dispersed indicating that temperature could play a hand in determining the cost of cab ride.

*Hover over the visualization to compare costs and control the visualization using the legend in the right corner.*
<br><br>

In [0]:
x=less3_df[less3_df['distance'] == 2.32].groupby(by='windSpeed')['price'].mean().index.values
y=less3_df[less3_df['distance'] == 2.32].groupby(by='windSpeed')['price'].mean().values
name='2.32 miles'
trace15a = plot_scatter_chart(x, y, name=name)

x=less6_df[less6_df['distance'] == 3.05].groupby(by='windSpeed')['price'].mean().index.values
y=less6_df[less6_df['distance'] == 3.05].groupby(by='windSpeed')['price'].mean().values
name='3.05 miles'
trace15b = plot_scatter_chart(x, y, name=name)

x=less15_df[less15_df['distance'] == 7.46].groupby(by='windSpeed')['price'].mean().index.values
y=less15_df[less15_df['distance'] == 7.46].groupby(by='windSpeed')['price'].mean().values
name='7.46 miles'
trace15c = plot_scatter_chart(x, y, name=name)

layout = layout_details('<b>Average Cost of Ride based on Wind Speed</b>', 
                        '<b><i>Wind Speed in MPH</i></b>', 
                        '<b><i>Cost in USD</i></b>')
display_trace([trace15a, trace15b, trace15c], layout)

The previous inference can be made for the above visualization as well where wind speed tends to affect costs of longer cab rides while costs of shorter rides are closely clustered together.<br><br>
*Hover over the visualization to compare costs and control the visualization using the legend in the right corner.*
<br><br>

In [0]:
x=less3_df[less3_df['distance'] == 2.32].groupby(by='precipIntensity')['price'].mean().index.values
y=less3_df[less3_df['distance'] == 2.32].groupby(by='precipIntensity')['price'].mean().values
name='2.32 miles'
trace16a = plot_scatter_chart(x, y, name=name)

x=less6_df[less6_df['distance'] == 3.05].groupby(by='precipIntensity')['price'].mean().index.values
y=less6_df[less6_df['distance'] == 3.05].groupby(by='precipIntensity')['price'].mean().values
name='3.05 miles'
trace16b = plot_scatter_chart(x, y, name=name)

x=less15_df[less15_df['distance'] == 7.46].groupby(by='precipIntensity')['price'].mean().index.values
y=less15_df[less15_df['distance'] == 7.46].groupby(by='precipIntensity')['price'].mean().values
name='7.46 miles'
trace16c = plot_scatter_chart(x, y, name=name)


layout = layout_details('<b>Average Cost of Ride based on Precipitation Intensity</b>', 
                        '<b><i>Intensity</i></b>', 
                        '<b><i>Cost in USD</i></b>')
display_trace([trace16a, trace16b, trace16c], layout)

Precipitation intensity too seems to affect costs for longer cab rides. However, the number of data points to back this conclusion is very low.<br><br>

*Hover over the visualization to compare costs and control the visualization using the legend in the right corner.*
<br><br>

## Machine Learning

In [0]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score, confusion_matrix, classification_report

In [0]:
def regression_stats(y_test, pred):
    '''Print regression metrics.'''
    print('r^2: ', round(r2_score(y_test, pred), 2))
    print('MAE: ', round(mean_absolute_error(y_test, pred), 2))
    print('RMSE: ', round(np.sqrt(mean_squared_error(y_test, pred)), 2), '\n')
    
def classification_stats(y_test, pred):
    '''Print classification metrics.'''
    print('Accuracy: ', round(accuracy_score(y_test, pred), 2), '\n')    
    print('Classification Report: \n', classification_report(y_test, pred), '\n')
    print('Confusion Matrix: \n', confusion_matrix(y_test, pred), '\n')
    
def fit_predict(model, X_train, y_train, X_test):
    '''Trains model and returns predictions on test set.'''
    print(model.fit(X_train, y_train), '\n')
    pred = model.predict(X_test)
    return pred

def create_pred_dataframe(y_test, pred):
    '''Returns dataframe with Actual value and Predicted value.'''
    return pd.DataFrame({'Actual': y_test, 'Predicted': pred})

def create_coeff_dataframe(model, X_train):
    '''Returns dataframe with coeffcients of Linear Regression model.'''
    return pd.DataFrame(model.coef_, X_train.columns, columns=['Coefficients'])

def create_feat_importance_dataframe(model, X_train):
    'Returns dataframe with feature importances.'
    return pd.DataFrame(model.feature_importances_, X_train.columns, columns=['Importance'])

In [0]:
df.columns

Index(['distance', 'cab_type', 'timestamp', 'source', 'destination', 'price',
       'surge_multiplier', 'product_id', 'name', 'cloudCover', 'temperature',
       'windSpeed', 'uvIndex', 'precipIntensityMax', 'precipIntensity',
       'precipProbability', 'hour', 'day', 'source_latitude',
       'source_longitude', 'destination_latitude', 'destination_longitude'],
      dtype='object')

In [0]:
#Separate dataframe for regression task.
reg_df = pd.get_dummies(df)
X = reg_df.drop(columns=['price']) 
y = reg_df['price'] #Target variable which is the cost of a ride.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42) #Train test split: 67:33

## Linear Regression

In [0]:
lin_reg = LinearRegression()
pred = fit_predict(lin_reg, X_train, y_train, X_test)
regression_stats(y_test, pred)
pred_df = create_pred_dataframe(y_test, pred)
coeff_df = create_coeff_dataframe(lin_reg, X_train)
print('Sample of Actual vs Predicted Dataframe:\n\n', pred_df.sample(10), '\n')
print('Top 10 Linear Regression Coefficients:\n\n',
      coeff_df.sort_values(by='Coefficients', ascending=False).head(10), '\n')

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False) 

r^2:  0.93
MAE:  1.75
RMSE:  2.48 

Sample of Actual vs Predicted Dataframe:

         Actual  Predicted
294329    16.0  15.950907
47071     22.5  23.027644
164495    15.0  16.254492
510177    10.5  12.189848
511453    15.5  16.356678
385484    13.5  15.276238
8957       7.0   4.461886
507986    11.5  10.379850
95255     10.5  11.814895
389456    39.5  36.923502 

Top 10 Linear Regression Coefficients:

                                                  Coefficients
surge_multiplier                                    19.698512
product_id_lyft_luxsuv                               7.512227
name_Lux Black XL                                    7.512227
name_Black SUV                                       7.246550
product_id_6d318bcc-22a3-4af6-bddd-b409bfce1546      7.246550
name_Lux Black                                       2.915863
product_id_lyft_lux                                  2.915863
distance      

## Random Forest Regressor

In [0]:
rfr = RandomForestRegressor()
pred = fit_predict(rfr, X_train, y_train, X_test)
regression_stats(y_test, pred)
pred_df = create_pred_dataframe(y_test, pred)
feat_imp_df = create_feat_importance_dataframe(rfr, X_train)
print('Sample of Actual vs Predicted Dataframe:\n\n', pred_df.sample(10), '\n')
print('Top 10 Important Features:\n\n',
      feat_imp_df.sort_values(by='Importance', ascending=False).head(10), '\n')

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False) 

r^2:  0.96
MAE:  1.15
RMSE:  1.85 

Sample of Actual vs Predicted Dataframe:

         Actual  Predicted
84061      9.0   9.000000
524451    22.5  24.600000
187007     7.5   7.066667
69312     26.5  27.300000
363955    26.0  26.900000
334516    27.5  26.450000
413040    27.5  29.500000
70929     26.0  26.750000
523987    23.0  22.600000
314752     9.5   8.950000 

Top 10 Important Features:

                                                  Importance
name_Lux Black XL                                  0.197695
distance           

In [0]:
#Separate dataframe for classification task.
X = df.drop(columns=['cab_type', 'product_id', 'name', 'timestamp'])
y = df['cab_type'] #Target variable which is company: Uber or Lyft
X = pd.get_dummies(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42) #Train test split: 67:33

## Logistic Regression

In [0]:
log_reg = LogisticRegression()
pred = fit_predict(log_reg, X_train, y_train, X_test)
classification_stats(y_test, pred)
pred_df = create_pred_dataframe(y_test, pred)
print('Sample of Actual vs Predicted Dataframe:\n\n', pred_df.sample(10), '\n')

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False) 

Accuracy:  0.55 

Classification Report: 
               precision    recall  f1-score   support

        Lyft       0.60      0.21      0.31     84618
        Uber       0.54      0.87      0.67     90806

    accuracy                           0.55    175424
   macro avg       0.57      0.54      0.49    175424
weighted avg       0.57      0.55      0.49    175424
 

Confusion Matrix: 
 [[17691 66927]
 [11893 78913]] 

Sample of Actual vs Predicted Dataframe:

        Actual Predicted
225434   Lyft      Uber
473165   Lyft      Uber
185028   Uber      Uber
362960   Uber      Uber
287656   Lyft      Uber
167765   Uber      Uber
200818   Lyft      Uber
93647    Lyft

## Random Forest Classifier

In [0]:
rfc = RandomForestClassifier()
pred = fit_predict(rfc, X_train, y_train, X_test)
classification_stats(y_test, pred)
pred_df = create_pred_dataframe(y_test, pred)
feat_imp_df = create_feat_importance_dataframe(rfc, X_train)
print('Sample of Actual vs Predicted Dataframe:\n\n', pred_df.sample(10), '\n')
print('Top 10 Important Features:\n\n',
      feat_imp_df.sort_values(by='Importance', ascending=False).head(10), '\n')

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False) 

Accuracy:  0.95 

Classification Report: 
               precision    recall  f1-score   support

        Lyft       0.94      0.95      0.94     84618
        Uber       0.95      0.94      0.95     90806

    accuracy                           0.95    175424
   macro avg       0.95      0.95      0.95    175424
weighted avg       0.95      0.95      0.95    175424
 

Confusion Matrix: 
 [[80527  4091]
 [ 5510 85296]] 

Sample of Actual vs Predicted Dataframe:

        Actual Predicted
324565   Uber   

## Conclusion

- Total Revenue: 
    - Uber: \$5.2 million
    - Lyft: \$5.3 million
<br><br>
- Price based on distance:
    - Overall, Uber is more affordable than Lyft.
    - Lyft is more affordable than Uber for economy rides.
    - Uber is more affordable than Lyft for luxury rides.
<br><br>
- Price based on time of day:
    - Cab rides for distances greater than 6 miles tend to vary throughout the day with the highest increase in price during rush hour 7am to 8am.
    - Lyft appears to modify their prices more than Uber based on time of day.
<br><br>    
- Price based on temperature, windspeed and precipitation:
    - Prices tend to vary more for distances with longer rides based on above mentioned external factors while prices for shorter rides are not affected by weather conditions.
